In [12]:
MARKET = 'SNP'

In [13]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [14]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [15]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [16]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [17]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
files

['dfrq.pkl',
 'df_chains.pkl',
 'df_ohlcs.pkl',
 'df_opts.pkl',
 'df_opt_margins.pkl',
 'df_opt_prices.pkl',
 'df_symlots.pkl',
 'df_unds.pkl',
 'df_und_margins.pkl',
 'df_und_prices.pkl',
 'qopts.pkl']

# Building a generic raw option preparer

In [18]:
THIS_FOLDER = ''
syms = ''            # If blank generate all
MINDTE = 0           # No minimum DTE. Use YAML MINDTE
MAXDTE = 7*30        # 7 month out
OVERLAPSDMULT = 2    # Put and Call SD overlap from edge

In [19]:
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

ibp = Vars(MARKET)

# If no symbols are given, use YAML defaults for dte
if not syms:
    MINDTE = ibp.MINDTE
    MAXDTE = ibp.MAXDTE
else:
    MINDTE = 0

In [19]:
# ... read the chains and unds
df_chains = pd.read_pickle(DATAPATH.joinpath("df_chains.pkl"))
df_unds = pd.read_pickle(DATAPATH.joinpath("df_unds.pkl"))

# ... weed out chains within MIN and outside MAX dte
df_ch = df_chains[
    df_chains.dte.between(MINDTE, MAXDTE, inclusive=True)
].reset_index(drop=True)

In [25]:
df_opts.sort_values('rom', ascending=False)

,conId,symbol,secType,localSymbol,strike,expiry,right,contract,time,model,bid,ask,close,last,sdMult,price,iv,undPrice,comm,margin,intrinsic,timevalue,dte,expiryM,lot,rom,undIV,undSD
16,450983599,XME,OPT,XME 201120P00023500,23.5,20201120,P,"Option(conId=450983599, symbol='XME', lastTrad...",2020-11-15 15:31:35.551268+00:00,None,-1.0,-1.0,0.06,NaN,NaN,0.06,NaN,26.49,1.585865,47.80,0.0,0.06,5,2020-11,100,6.741253,0.339768,2.838363
11,446015346,XME,OPT,XME 201120P00023000,23.0,20201120,P,"Option(conId=446015346, symbol='XME', lastTrad...",2020-11-15 15:31:35.551268+00:00,None,-1.0,-1.0,0.03,NaN,NaN,0.03,NaN,26.49,1.585846,47.80,0.0,0.03,5,2020-11,100,2.159692,0.339768,3.313006
45,418439265,XME,OPT,XME 201218P00018000,18.0,20201218,P,"Option(conId=418439265, symbol='XME', lastTrad...",2020-11-15 15:31:35.551268+00:00,None,-1.0,-1.0,0.07,NaN,NaN,0.07,NaN,26.49,1.585877,37.50,0.0,0.07,33,2020-12,100,1.596893,0.339768,3.137130
4,418439256,XME,OPT,XME 201218P00017000,17.0,20201218,P,"Option(conId=418439256, symbol='XME', lastTrad...",2020-11-15 15:31:35.551268+00:00,None,-1.0,-1.0,0.05,NaN,NaN,0.05,NaN,26.49,1.585855,37.50,0.0,0.05,33,2020-12,100,1.007000,0.339768,3.506638
47,446015176,XME,OPT,XME 201120C00029000,29.0,20201120,C,"Option(conId=446015176, symbol='XME', lastTrad...",2020-11-15 15:31:35.551268+00:00,None,-1.0,-1.0,0.05,NaN,NaN,0.05,NaN,26.49,1.586035,469.45,0.0,0.05,5,2020-11,100,0.530875,0.339768,2.382706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,418439280,XME,OPT,XME 201218P00001000,1.0,20201218,P,"Option(conId=418439280, symbol='XME', lastTrad...",2020-11-15 15:31:34.530869+00:00,None,-1.0,-1.0,0.00,NaN,NaN,0.00,NaN,26.49,1.585800,37.50,0.0,0.00,33,2020-12,100,0.000000,0.339768,9.418779
41,455068871,XME,OPT,XME 201211C00034000,34.0,20201211,C,"Option(conId=455068871, symbol='XME', lastTrad...",2020-11-15 15:31:35.551268+00:00,None,-1.0,-1.0,0.01,NaN,NaN,0.01,NaN,26.49,1.585844,109.78,0.0,0.01,26,2020-12,100,0.000000,0.339768,3.126331
40,418439205,XME,OPT,XME 201218P00011000,11.0,20201218,P,"Option(conId=418439205, symbol='XME', lastTrad...",2020-11-15 15:31:34.530869+00:00,None,-1.0,-1.0,0.00,NaN,NaN,0.00,NaN,26.49,1.585800,37.50,0.0,0.00,33,2020-12,100,0.000000,0.339768,5.723691
39,451578273,XME,OPT,XME 201204P00018500,18.5,20201204,P,"Option(conId=451578273, symbol='XME', lastTrad...",2020-11-15 15:31:34.530869+00:00,None,-1.0,-1.0,0.00,NaN,NaN,0.00,NaN,26.49,1.585800,37.50,0.0,0.00,19,2020-12,100,0.000000,0.339768,3.890915


In [26]:
dfrq[dfrq.symbol == 'XME']

,symbol,undPrice,lot,gross,grosspos,remq,status
227,XME,27.74,100,2649.0,8400.0,5,orphans
